<a href="https://colab.research.google.com/github/mirpouya/Transformer_EDU/blob/main/Vision_Transformer_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets

In [5]:
!pip install datasets transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00


In [9]:
# import CIFAR-10 dataset from HuggingFace
from datasets import load_dataset

# train split
dataset_train = load_dataset(
    "cifar10",
    split = "train",
    ignore_verifications = False  # set to True if seeing splits error
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1760: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=all_checks' instead.
  warnings.warn(


Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/features/image.py:325: UserWarning: Downcasting array dtype uint8 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4. Subsequent calls will reuse this data.


In [10]:
dataset_train

Dataset({
    features: ['img', 'label'],
    num_rows: 50000
})

In [12]:
# test split
dataset_test = load_dataset(
    "cifar10",
    split = "test",
    ignore_verifications = True
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1760: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


In [13]:
dataset_test

Dataset({
    features: ['img', 'label'],
    num_rows: 10000
})

In [14]:
# check how many labels we have - number of classes
num_classes = len(set(dataset_test["label"]))
labels = dataset_test.features["label"]

num_classes, labels

(10,
 ClassLabel(names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], id=None))

In [15]:
dataset_train[0]

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7FAA1CF57850>,
 'label': 0}

In [16]:
dataset_train[0]["label"], labels.names[dataset_train[0]["label"]]

(0, 'airplane')

## <b> Loading Vision Transformer Feature Extractor </b>

In [17]:
from transformers import ViTFeatureExtractor

In [18]:
# import model
model_name_or_path = "google/vit-base-patch16-224-in21k"

feature_extractor = ViTFeatureExtractor.from_pretrained(
    model_name_or_path
)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [19]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [33]:
# testing feature extractor
example = feature_extractor(
    dataset_train[0]["img"],
    return_tensors = "pt"
)

In [21]:
example

{'pixel_values': [array([[[ 0.39607847,  0.39607847,  0.39607847, ...,  0.2941177 ,
          0.2941177 ,  0.2941177 ],
        [ 0.39607847,  0.39607847,  0.39607847, ...,  0.2941177 ,
          0.2941177 ,  0.2941177 ],
        [ 0.39607847,  0.39607847,  0.39607847, ...,  0.2941177 ,
          0.2941177 ,  0.2941177 ],
        ...,
        [-0.19215685, -0.19215685, -0.19215685, ..., -0.2862745 ,
         -0.2862745 , -0.2862745 ],
        [-0.19215685, -0.19215685, -0.19215685, ..., -0.2862745 ,
         -0.2862745 , -0.2862745 ],
        [-0.19215685, -0.19215685, -0.19215685, ..., -0.2862745 ,
         -0.2862745 , -0.2862745 ]],

       [[ 0.3803922 ,  0.3803922 ,  0.3803922 , ...,  0.27843142,
          0.27843142,  0.27843142],
        [ 0.3803922 ,  0.3803922 ,  0.3803922 , ...,  0.27843142,
          0.27843142,  0.27843142],
        [ 0.3803922 ,  0.3803922 ,  0.3803922 , ...,  0.27843142,
          0.27843142,  0.27843142],
        ...,
        [-0.24705881, -0.24705881, -

In [30]:
import numpy as np
np.asarray(example["pixel_values"]).shape

(1, 3, 224, 224)

In [31]:
# original shape of image
dataset_train[0]["img"].size

(32, 32)

### <b> Switch to GPU </b>

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [34]:
# preprocessing

def preprocessing(batch):
  # takes a list of PIL images and change them to pixel values
  inputs = feature_extractor(
      batch["img"],
      return_tensors = "pt"
  ).to(device)

  # including the labels
  inputs["labels"] = batch["lebel"]
  return inputs

In [35]:
# collate function to create batch tensors
def collate_fn(batch):

  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["label"] for x in batch])
  }